<a href="https://colab.research.google.com/github/ouguro3/Study/blob/main/ML_Guide/chapter_2/02_Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 모듈 소개

사이킷런의 model_selection 모듈은 학습 데이터와 테스트 데이터 세트를 분리하거나  
교차 검증 분할 및 평가, 그리고 Estimator의 하이퍼 파라미터를 튜닝하기 위한  
다양한 함수와 클래스를 제공한다

### 학습/테스트 데이터 세트 분리 - train_test_split()

사이킷런의 `train_test_split()`을 통해 원본 데이터 세트에서  
학습 및 테스트 데이터 세트를 쉽게 분리할 수 있다

train_test_split()은 첫 번째 파라미터로 피처 데이터 세트, 두 번째 파라미터로 레이블 데이터 세트를 입력받는다  

그리고 선택적으로 다음 파라미터를 입력받는다  

- test_size (기본값 : 0.25) : 전체 데이터에서 테스트 데이터 세트 크기를 얼마로 샘플링할 것인가를 결정한다  

- train_size : 전체 데이터에서 학습용 데이터 세트 크기를 얼마로 샘플링할 것인가를 결정한다  
다만 통상적으로 `test_size`를 사용하므로 잘 쓰이지 않는다  

- shuffle (기본값 : True) : 데이터를 분리하기 전에 데이터를 미리 섞을지를 결정한다  
데이터를 분산시켜서 좀 더 효율적인 학습 및 테스트 데이터 세트를 만드는 데 사용된다 

- random_state : 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수값이다  
호출 시 무작위로 데이터를 분리하므로 지정하지 않으면 수행할 때마다 다른 학습/테스트용 데이터를 생성한다

In [9]:
# 붓꽃 데이터 세트를 train_test_split()을 사용해 테스트 데이터 30% 학습 데이터 70%으로 분리

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size=0.3, random_state=121)

In [10]:
# 예측 정확도 측정
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도: 0.9556


### 교차 검증

알고리즘을 학습시키는 학습 데이터와 이에 대한 예측 성능을 평가하기 위한 별도의 테스트 데이터가 필요한데  
이 방법 역시 과적합에 취약한 약점을 가질 수 있다  

과적합은 모델이 특정 학습 데이터에만 과도하게 최적화되어 다른 데이터에는  
예측 성능이 떨어지는것을 말한다  

이런 문제점을 개선하기 위해 교차 검증을 이용해 더 다양한 학습과 평가를 수행한다

#### K 폴드 교차 검증

K 폴드 교차 검증은 가장 보편적으로 사용되는 교차 검증 기법이다  

먼저 K 개의 데이터 폴드 세트를 만들어서 K 번만큼 각 폴드 세트에 학습과 검증 평가를   
반복적으로 수행하는 방법이다  

K 개의 예측 평가를 구했으면 이를 평균해서 K 폴드 평가 결과로 반영한다

In [11]:
# K 폴드 붓꽃 데이터 예측 평가
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기 :', features.shape[0])

붓꽃 데이터 세트 크기 : 150


`KFold = (n_splits=5)` 로 KFold 객체를 생성했으니 생성된 객체의 split()을 호출해  
전체 붓꽃 데이터를 5개의 폴드 데이터 세트로 분리한다  

전체 붓꽃 데이터는 150개이며, 이중 학습용 데이터는 4/5인 120개,  
검증 테스트 데이터 세트는 1/5인 30개로 분할된다  

KFold 객체는 split()을 호출하면 학습용/검증용 데이터로 분할할 수 있는 인덱스를 반환한다  

In [12]:
# 5개의 폴드 세트를 생성하는 KFold 객체의 split()을 호출, 
# 교차 검증 수행 시마다 학습과 검증을 반복해 예측 정확도를 측정한다

n_iter = 0

# KFold 객체의 split()을 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
  # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
  X_train, X_test = features[train_index], features[test_index]
  y_train, y_test = label[train_index], label[test_index]
  # 학습 및 예측
  dt_clf.fit(X_train,y_train)
  pred = dt_clf.predict(X_test)
  n_iter += 1
  # 반복 시마다 정확도 측정
  accuracy = np.round(accuracy_score(y_test, pred), 4)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]
  print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'
        .format(n_iter, accuracy, train_size, test_size))
  print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
  cv_accuracy.append(accuracy)

# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도 :',np.mean(cv_accuracy))


#1 교차 검증 정확도 :1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도 : 0.9


#### Stratified K 폴드

Stratified K 폴드는 불균형한 분포도를 가진 레이블(결정 클래스) 데이터 집합을 위한 K 폴드 방식이다  

불균형한 분포도를 가진 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 매우 적어서  
값의 분포가 한쪽으로 치우치는 것을 말한다  

Startified K 폴드는 이처럼 K 폴드가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를  
학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제를 해결해준다  

이를 위해 Stratified K 폴드는 원본 데이터의 레이블 분포를 먼저 고려한 뒤  
이 분포와 동일하게 학습과 검증 데이터 세트를 분배한다  

In [13]:
# K 폴드의 문제 확인 후 StratifiedKFold 를 이용해 개선시키기

import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data= iris.data, columns=iris.feature_names)
iris_df['label']= iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [14]:
# 레이블 값은 0,1,2 값 모두 50개로 동일
# Setosa, Versicolor, Virginica 모두 50개

# 3개의 폴드 세트를 KFold로 생성하고, 각 교차 검증 시마다 생성되는
# 학습/검증 레이블 데이터 값의 분포도를 확인

kfold = KFold(n_splits=3)
n_iter=0
for train_index, test_index in kfold.split(iris_df):
  n_iter += 1
  label_train = iris_df['label'].iloc[train_index]
  label_test = iris_df['label'].iloc[test_index]
  print('## 교차 검증 : {0}'.format(n_iter))
  print('학습 레이블 데이터 분포 :\n', label_train.value_counts())
  print('검증 레이블 데이터 분포 :\n', label_test.value_counts())

## 교차 검증 : 1
학습 레이블 데이터 분포 :
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 0    50
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포 :
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    50
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포 :
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    50
Name: label, dtype: int64


첫 번째 교차 검증에서는 학습 레이블의 1, 2 값이 각각 50개가 추출되었고,  
검증 레이블의 0 값이 50개 추출되었다  

학습 레이블은 1, 2 밖에 없으므로 0의 경우는 학습하지 못하고,  

반대로 검증 레이블은 0밖에 없으므로 학습 모델은 절대 0을 예측하지 못한다  

이런 유형으로 교차 검증 데이터 세트를 분할하면 검증 예측 정확도는 0이 될 수 밖에 없다

StratifiedKFold는 이렇게 KFold로 분할된 레이블 데이터 세트가 전체 레이블 값의  
분포도를 반영하지 못하는 문제를 해결해 준다  

StratifiedKFold의 사용법은 KFold 사용법과 거의 비슷하다  

차이점은 StratifiedKFold는 레이블 데이터 분포도에 따라 학습/검증 데이터를 나누기 때문에  
split() 메서드에 인자로 피처 데이터 세트뿐만 아니라 레이블 데이터 세트도 반드시 필요하다는 사실이다  
(K 폴드의 경우 레이블 데이터 세트는 split() 메서드의 인자로 입력하지 않아도 무방하다) 

In [15]:
# 폴드 세트는 3개로 설정

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
  n_iter += 1
  label_train = iris_df['label'].iloc[train_index]
  label_test = iris_df['label'].iloc[test_index]
  print('## 교차 검증 : {0}'.format(n_iter))
  print('학습 레이블 데이터 분포 :\n', label_train.value_counts())
  print('검증 레이블 데이터 분포 :\n', label_test.value_counts())

## 교차 검증 : 1
학습 레이블 데이터 분포 :
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포 :
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포 :
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    17
1    17
0    16
Name: label, dtype: int64


In [17]:
# StratifiedKFold를 이용해 데이터 분리

df_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy=[]

# StratifiedKFold의 split() 호출시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_index in skfold.split(features, label):
  # split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
  X_train, X_test = features[train_index], features[test_index]
  y_train, y_test = label[train_index], label[test_index]
  # 학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_test)

  # 반복 시마다 정확도 측정
  n_iter += 1
  accuracy = np.round(accuracy_score(y_test, pred), 4)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]
  print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'
        .format(n_iter, accuracy, train_size, test_size))
  print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
  cv_accuracy.append(accuracy)
  # 교차 검증별 정확도 및 평균 정확도 계산
  print('\n## 교차 검증별 정확도 :', np.round(cv_accuracy, 4))
  print('## 평균 검증 정확도 :', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

## 교차 검증별 정확도 : [0.98]
## 평균 검증 정확도 : 0.98

#2 교차 검증 정확도 : 0.94, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#2 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

## 교차 검증별 정확도 : [0.98 0.94]
## 평균 검증 정확도 : 0.96

#3 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#3 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도 : [0.98 0.94 0.98]
## 평균 검증 정확도 : 0.9666666666666667


사실, 일반적으로 분류(Classification) 에서의 교차 검증은 K 폴드가 아니라  
Stratified K 폴드로 분할돼어야 한다  

회귀(Regression)에서의 교차 검증은 K 폴드가 지원되지 않는다  

이유는 회귀의 결정값은 이산값 형태의 레이블이 아니라 연속된 숫자값이기 때문에  
결정값별로 분포를 정하는 의미가 없기 때문이다  

#### 교차 검증을 보다 간편하게 - cross_val_score()

사이킷런에서 제공하는 교차 검증을 좀 더 편하게 수행할 수 있게 해주는 API  

KFold로 데이터를 학습하고 예측하는 코드를 보면
1. 폴드 세트를 설정
2. for 루프에서 반복으로 학습 및 테스트 데이터의 인덱스 추출
3. 반복적으로 학습 및 예측을 수행하고 예측 성능을 반환  

일련의 과정을 거친다

`cross_val_score()`은 이런 과정을 한꺼번에 수행해주는 API 이다

cross_val_score()의 선언 형태는 아래와 같다
```
cross_val_score(estimator, X, y=None, scoring=None, cv=None, n_jobs=1,  
verbose=0, fit_params=None, pre_dispatch='2*n_jobs')
```
이 중 estimator, X, y, scoring, cv 가 주요 파라미터이다  

- estimator : 사이킷런의 분류 알고리즘 Classifier, 또는 회귀 알고리즘 Regressor를 의미한다  

- X : 피처 데이터 세트

- y : 레이블 데이터 세트

- scoring : 예측 성능 평가 지표

- cv : 교차 검증 폴드 수  

cross_val_score()는 classifier가 입력된 경우 StratifiedKFold 방식으로  
레이블값의 분포에 따라 학습/데이터 세트를 분할한다  
(Regressor의 경우 KFold 방식으로 분할)

In [18]:
# cross_val_score() 사용법
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도 :',np.round(scores, 4))
print('교차 검증 정확도 :',np.round(np.mean(scores), 4))

교차 검증별 정확도 : [0.98 0.94 0.98]
교차 검증 정확도 : 0.9667


### GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에

하이퍼 파라미터를 순차적으로 입력하면서 최적의 파라미터를 도출할 수 있는 방안을 제공한다

In [ ]:
grid_paramaters = {'max_depth' : [1,2,3],
                   'min_samples_split' : [2,3]}

GridSearchCV의 주요 파라미터

- estimator : classifier, regressor, pipeline

- param_grid : key + 리스트 값을 가지는 딕셔너리, estimator의 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정한다  

- scoring : 예측 성능을 측정할 평가 방법을 지정

- cv : 교차 검증을 위해 분할되는 학습/테스트 세트의 개수를 지정

- refit : 디폴트가 True 이며 True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤  
입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다

In [23]:
# GridSearchCV의 사용법
# max_depth와 min_samples_split의 값을 변화시키면서 최적화 진행

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩하고 학습 데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

### 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [25]:
import pandas as pd

# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정
### refit=True가 default,  True이면 가장 좋은 파라미터 설정으로 재학습
grid_dtree = GridSearchCV(dtree, param_grid=parameters,cv=3,refit=True)

# 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score',
           'split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


- params : 수행할 때마다 적용된 개별 하이퍼 파라미터값을 나타낸다

- rank_test_score : 하이퍼 파라미터별로 성능이 좋은 score 순위

- mean_test_score : cv의 폴딩 테스트 세트에 대해 총 수행한 평가 평균값

GridSearchCV 의 fit()을 수행하면 최고 성능을 나타낸 하이퍼 파라미터의 값과  
그때의 평가 결과 값이 각각 `best_params_`, `best_score_` 속성에 기록된다

In [27]:
print('GridSearchCV 최적 파라미터 :', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터 : {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도 : 0.9750


In [28]:
# GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습이 필요 없음
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도 : {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도 : 0.9667
